In [1]:
import sys
# 'sys.path' is a list of absolute path strings
sys.path.append('/home/ricky/rkw0k/PyPortfolioOpt')

In [4]:
import logging
import pandas as pd
import numpy as np
import yfinance as yf
from pypfopt import risk_models
from pypfopt import plotting
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict

logger = logging.getLogger(__name__)
logger.addHandler(logging.NullHandler())

def get_historical_prices(tickers, start="1970-01-01"):
    df = yf.download(tickers, start, progress=False, auto_adjust=True)
    # keep rows where at least one ticker has a price (avoid removing earlier history)
    df_prices = df["Close"].dropna(how="all")
    logger.info("Historical data: %s -> %s, %d trading days",
                df_prices.index.min().strftime("%Y-%m-%d"),
                df_prices.index.max().strftime("%Y-%m-%d"),
                len(df_prices))
    return df_prices

def backtest_portfolio(df_prices, target_weights, initial_value=1_000_000, start_year=2000, swr=0.05):
    df_monthly = df_prices.resample("ME").last()
    df_monthly_returns = df_monthly.pct_change().dropna()
    tw = pd.Series(target_weights)
    portfolio_value = initial_value
    asset_values = (tw * portfolio_value).to_dict()
    history = []

    for year, group in df_monthly_returns.groupby(df_monthly_returns.index.year):
        start_val = portfolio_value
        withdrawal = initial_value * swr * (1.03 ** (year - start_year))
        portfolio_value -= withdrawal
        for t, w in tw.items():
            asset_values[t] = portfolio_value * w
        for date in group.index:
            returns = group.loc[date]
            for ticker in list(asset_values.keys()):
                asset_values[ticker] *= (1 + returns[ticker])
            portfolio_value = sum(asset_values.values())
        history.append({
            "Year": year,
            "Start Value (Pre-Withdrawal)": start_val,
            "Withdrawal Amount": withdrawal,
            "End Value (Post-Growth)": portfolio_value,
        })

    df_history = pd.DataFrame(history).set_index("Year")
    df_history["Net Return (%)"] = (df_history["End Value (Post-Growth)"] /
                                   (df_history["Start Value (Pre-Withdrawal)"] - df_history["Withdrawal Amount"]) - 1) * 100
    return df_history

def monte_carlo_historical(historical_data, num_trials, simulation_years, verbose=False):
    if isinstance(historical_data, pd.DataFrame):
        price_series = historical_data.mean(axis=1)
    else:
        price_series = historical_data.copy()
    price_series = price_series.dropna()

    last_start = price_series.index.max() - pd.DateOffset(years=simulation_years)

    first_trading_of_year = price_series.groupby(price_series.index.year).apply(lambda g: g.index.min())
    first_trading_of_year = pd.DatetimeIndex(first_trading_of_year.values)
    valid_starts = first_trading_of_year[first_trading_of_year <= last_start]

    if valid_starts.empty:
        candidate = price_series[price_series.index <= last_start].index
        if candidate.empty:
            raise ValueError(
                "Insufficient history for requested simulation_years; reduce years or provide more data."
            )
        valid_starts = candidate

    if verbose:
        logger.info("Monte Carlo: %d trials, valid starts %s -> %s",
                    num_trials, valid_starts.min().date(), valid_starts.max().date())

    results = []
    for i in range(1, num_trials + 1):
        sd = pd.Timestamp(np.random.choice(valid_starts))
        ed = sd + pd.DateOffset(years=simulation_years)
        path = price_series.loc[sd:ed]
        if len(path) < 2:
            continue
        normalized = (path / path.iloc[0]) * 100
        # use integer offset index for consistent alignment, keep name
        results.append(pd.Series(normalized.values, index=np.arange(len(normalized)),
                                 name=f"Trial_{i}_Start_{sd.year}"))

    if not results:
        raise ValueError("No valid trials generated from the historical_data provided.")

    # trim all trials to the shortest path to avoid NaNs from mismatched dates
    min_len = min(len(s) for s in results)
    trimmed = [s.iloc[:min_len] for s in results]
    results_df = pd.concat(trimmed, axis=1)
    return results_df

def summarize_simulation(sim_df, initial_value=1_000_000, swr=0.04,
                         simulation_years=None, trading_days_per_year=252):
    """Return summary stats for each Monte Carlo trial in sim_df.

    sim_df: DataFrame with trials as columns and normalized values (start=100).
    """
    if simulation_years is None:
        simulation_years = max(1, int(round(len(sim_df) / trading_days_per_year)))

    def total_withdrawals(initial, years, swr_rate):
        n = max(1, int(np.floor(years)))
        return sum(initial * swr_rate * (1.03 ** i) for i in range(n))

    rows = []
    for col in sim_df.columns:
        vals = sim_df[col] / 100.0 * initial_value
        if vals.empty:
            continue
        end_val = float(vals.iloc[-1])
        years = float(simulation_years)
        cagr = (end_val / initial_value) ** (1 / years) - 1 if years > 0 else np.nan

        daily_ret = vals.pct_change().dropna()
        vol_annual = float(daily_ret.std() * np.sqrt(trading_days_per_year)) if not daily_ret.empty else np.nan

        running_max = vals.cummax()
        max_dd = float((vals / running_max - 1).min()) if not vals.empty else 0.0

        rows.append({
            "Trial": col,
            "End Value": end_val,
            "Total Withdrawals": total_withdrawals(initial_value, years, swr),
            "Largest Drawdown (%)": abs(max_dd) * 100,
            "Volatility (%)": vol_annual * 100,
            "Years": years,
            "CAGR (%)": cagr * 100,
        })

    summary_df = pd.DataFrame(rows).set_index("Trial")
    # add quartiles and mean as optional aggregated row
    agg = summary_df.agg(['mean', 'median', 'min', 'max'])
    return summary_df, agg

def choose_long_history_tickers(required_years=30):
    groups = {
        "stocks": ["^GSPC", "SPY", "VTI"],
        "treasury": ["^TYX", "^TNX", "TLT", "IEF"],
        "gold": ["GC=F", "GLD"],
    }
    selected = []
    for _, candidates in groups.items():
        best = None
        best_start = pd.Timestamp.max
        for t in candidates:
            try:
                s = yf.download(t, start="1900-01-01", progress=False, auto_adjust=True)["Close"].dropna()
            except Exception:
                s = pd.Series(dtype=float)
            if not s.empty:
                if s.index.min() < best_start:
                    best_start = s.index.min()
                    best = t
                if s.index.min() <= (s.index.max() - pd.DateOffset(years=required_years)):
                    best = t
                    break
        if best is None:
            best = min(candidates, key=lambda t: (
                yf.download(t, start="1900-01-01", progress=False, auto_adjust=True)["Close"].dropna().index.min()
                if not yf.download(t, start="1900-01-01", progress=False, auto_adjust=True)["Close"].dropna().empty
                else pd.Timestamp.max
            ))
            logger.warning("No candidate had %dyr history; selected best available: %s", required_years, best)
        selected.append(best)
    return selected



if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

    SIM_YEARS = 45
    NUM_TRIALS = 1000
    # choose tickers that have at least SIM_YEARS of history where possible
    TICKERS = choose_long_history_tickers(SIM_YEARS)
    weights_dict = OrderedDict()
    weights_dict["GC=F"] = 0.3
    weights_dict["^GSPC"] = 0.3
    weights_dict["^TYX"] = 0.4
    TICKERS = list(weights_dict.keys())
    logger.info("Selected tickers for simulation: %s", TICKERS)

    df_prices = get_historical_prices(TICKERS)

    # make SIM_YEARS robust: if combined data isn't old enough, lower SIM_YEARS
    available_years = int((df_prices.index.max() - df_prices.index.min()).days / 365)
    if available_years < SIM_YEARS:
        logger.warning("Requested SIM_YEARS=%d but only %d years available; reducing simulation years.",
                       SIM_YEARS, available_years)
        SIM_YEARS = max(1, available_years)

    starting_value = 4_000_000
    swr = 0.055
    if weights_dict:
        weights_series = pd.Series(weights_dict)

        logger.info("|".join(weights_series.to_string().split("\n")))
        df_backtest = backtest_portfolio(
            df_prices, weights_dict, starting_value, df_prices.index.min().year, swr
        )
        logger.info("%d years backtested", len(df_backtest))
    else:
        logger.warning("Could not calculate weights; skipping backtest.")

    sim_results = monte_carlo_historical(df_prices, NUM_TRIALS, SIM_YEARS, verbose=True)
    summary_df, agg = summarize_simulation(sim_results, starting_value, swr)
    logger.info("Simulation summary rows: %d", len(summary_df))
    logger.info("Aggregates: %s", agg.to_dict())
plotting.plot_simulation_results(sim_results, initial_value=4_000_000)

INFO: Selected tickers for simulation: ['GC=F', '^GSPC', '^TYX']
INFO: Historical data: 1970-01-02 -> 2025-11-18, 14097 trading days
INFO: GC=F     0.3|^GSPC    0.3|^TYX     0.4
INFO: 26 years backtested
INFO: Monte Carlo: 1000 trials, valid starts 1970-01-02 -> 1980-01-02
INFO: Simulation summary rows: 1000
INFO: Aggregates: {'End Value': {'mean': 89008904.01218863, 'median': 81916010.11069617, 'min': 44200683.19168347, 'max': 196160532.9981723}, 'Total Withdrawals': {'mean': 20398369.505443197, 'median': 20398369.505443197, 'min': 20398369.505443197, 'max': 20398369.505443197}, 'Largest Drawdown (%)': {'mean': 78.99323755933597, 'median': 78.99323755933594, 'min': 78.99323755933592, 'max': 78.99323755933594}, 'Volatility (%)': {'mean': 130.51139376872754, 'median': 132.20559520455123, 'min': 122.58505265988357, 'max': 132.2557245510372}, 'Years': {'mean': 45.0, 'median': 45.0, 'min': 45.0, 'max': 45.0}, 'CAGR (%)': {'mean': 6.776516235927532, 'median': 6.9400038862461155, 'min': 5.48

In [5]:
plotting.plot_simulation_results(sim_results, initial_value=4_000_000)

AttributeError: module 'pypfopt.plotting' has no attribute 'plot_simulation_results'

In [ ]:
start_year